In [51]:
import torch
import torchvision

In [52]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [53]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [54]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [55]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [56]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [57]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [58]:
def train(epoch, network, optimizer):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), 'model.pth')
      torch.save(optimizer.state_dict(), 'optimizer.pth')

In [59]:
def test(network):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [60]:
results = []
for i in range(10):
  torch.manual_seed(i)
  network = Net()
  optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
  test(network)
  for epoch in range(1, n_epochs + 1):
    train(epoch, network, optimizer)
    test(network)
  results.append(network)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3054, Accuracy: 767/10000 (8%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.296218
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.312881
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.309128
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.273404
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.271440
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.239971
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.243802
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.186078
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.224105
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.155078
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.100496
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.945586
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.017599
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.947806
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.670207
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.660531
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.463048
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.379897
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.486553
Train Epoch: 1 [12160/6

In [61]:
print(results)

[Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
), Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
), Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
), Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5